In [1]:
import pandas as pd
import pulp
import time

# Einlesen aller Tabellen
game = pd.read_csv('data/bc_game.csv')
streaming_offer = pd.read_csv('data/bc_streaming_offer.csv')
streaming_packages = pd.read_csv('data/bc_streaming_package.csv')

In [2]:
# Funktion mit Rückgabe aller Spiele für ein Team
def get_game_ids(team_list):
    filtered_games = game[game['team_home'].isin(team_list) | game['team_away'].isin(team_list)]
    return filtered_games['id'].tolist()

In [3]:
# Extrahierung aller nötigen Daten für das LpProblem
anbieter = streaming_packages["id"].tolist()
preis = dict(zip(streaming_packages["id"], streaming_packages["monthly_price_yearly_subscription_in_cents"]))
abdeckung = streaming_offer[streaming_offer["live"] == 1].groupby("streaming_package_id")["game_id"].apply(list).to_dict()

In [4]:
def ILP_set_cover(game_ids, anbieter, abdeckung, preis):
    """
    Greedy-Algorithmus für das Set Cover Problem.
    
    :param game_ids: Liste der Spiele, die abgedeckt werden müssen
    :param anbieter: Liste der Anbieter
    :param abdeckung: Dictionary mit Anbieter als Schlüssel und Liste der abgedeckten Spiele als Wert
    :param preis: Dictionary mit Anbieter als Schlüssel und den Kosten als Wert
    :return: Liste der ausgewählten Anbieter, Preis und ungedeckte Spiele
    """
    covered_games = [g for g in game_ids if any(g in abdeckung[k] for k in abdeckung)]
    uncovered_games = [g for g in game_ids if not any(g in abdeckung[k] for k in abdeckung)]

    # Das Set-Cover Problem wird als Lineares Programmierungsproblem definiert
    problem = pulp.LpProblem("MinimalCostPackages", pulp.LpMinimize)

    #Entscheidungsvariablen: Alle Anbieter ob diese gekauft werden oder nicht
    x = pulp.LpVariable.dicts('Anbieter', anbieter, cat='Binary')

    #Zielminimierung: Möglichst geringe kosten, 0/1 ob anbieter ausgewählt wird * preis
    problem += pulp.lpSum(preis[k]*x[k] for k in anbieter)

    #Einschränkungen: Jedes Spiel muss ansehbar sein wenn es auch angeboten wird
    for g in covered_games:
        problem += pulp.lpSum([
            x[k] for k in anbieter if k in abdeckung and g in abdeckung[k]
        ]) >= 1, f"Deckung_{g}"

    problem.solve() # Hier wird Branch-and-Cut verwendet

    selected_provider = []

    for a in anbieter:
        if pulp.value(x[a]) == 1:
            selected_provider.append(int(streaming_packages.loc[streaming_packages["id"] == a, "id"].iloc[0]))

    # Kosten minimieren
    price = pulp.value(problem.objective)

    return {"ausgewählte_anbieter": selected_provider, "preis": price, "ungedeckte_spiele": uncovered_games}

In [5]:
def greedy_set_cover(game_ids, anbieter, abdeckung, preis):
    """
    Greedy-Algorithmus für das Set Cover Problem.
    
    :param game_ids: Liste der Spiele, die abgedeckt werden müssen
    :param anbieter: Liste der Anbieter
    :param abdeckung: Dictionary mit Anbieter als Schlüssel und Liste der abgedeckten Spiele als Wert
    :param preis: Dictionary mit Anbieter als Schlüssel und den Kosten als Wert
    :return: Liste der ausgewählten Anbieter
    """
    covered_games = [g for g in game_ids if any(g in abdeckung[k] for k in abdeckung)]
    uncovered_games = [g for g in game_ids if not any(g in abdeckung[k] for k in abdeckung)]

    open_games = set(covered_games)  # Noch nicht abgedeckte Spiele
    selected_providers = []          # Ausgewählte Anbieter
    
    while open_games:
        # Finde den Anbieter mit dem besten Preis-Leistungs-Verhältnis
        best_provider = max(
            anbieter,
            key=lambda a: (
                len(open_games & set(abdeckung.get(a, []))) / preis[a] if preis[a] > 0 else 0
            )
        )
        
        # Füge den Anbieter zur Lösung hinzu
        selected_providers.append(best_provider)
        
        # Entferne die von diesem Anbieter abgedeckten Spiele aus der Liste
        open_games -= set(abdeckung.get(best_provider, []))

    price = 0
    for p in selected_providers:
        price += preis[p]

    return {'ausgewählte_anbieter': selected_providers, 'preis': price,'ungedeckte_spiele': uncovered_games}


In [8]:
game_ids = []
game_ids.append(get_game_ids(["Deutschland", "Hatayspor", "Bayern München", "Real Madrid"]))
game_ids.append(get_game_ids(["VfB Stuttgart"]))
game_ids.append(get_game_ids(["Oxford United", "Los Angeles FC", "AS Rom"]))
game_ids.append(get_game_ids(game[['team_home', 'team_away']].values.ravel()))

for entry in game_ids:
    optimum_time_start = time.time()
    optimum = ILP_set_cover(entry, anbieter, abdeckung, preis)
    optimum_time_end = time.time()
    greedy_time_start = time.time()
    greedy = greedy_set_cover(entry, anbieter, abdeckung, preis)
    greedy_time_end = time.time()

    print("Ausgewählte Anbieter von ILP: ", optimum["ausgewählte_anbieter"])
    print("Ausgewählte Anbieter von greedy: ", greedy["ausgewählte_anbieter"])
    print("Benötigte Zeit von ILP: ", optimum_time_end-optimum_time_start)
    print("Benötigte Zeit von greedy: ", greedy_time_end- greedy_time_start)
    if greedy["preis"] > optimum["preis"]:
        print("Der Greedy-Algorithmus gibt höhren Preis zurück!")
    print()

Ausgewählte Anbieter von ILP:  [2, 3, 4, 13, 16, 17, 43, 44, 51]
Ausgewählte Anbieter von greedy:  [51, 2, 17]
Benötigte Zeit von ILP:  0.13907814025878906
Benötigte Zeit von greedy:  0.04169106483459473

Ausgewählte Anbieter von ILP:  [2, 13, 17, 44]
Ausgewählte Anbieter von greedy:  [2, 17]
Benötigte Zeit von ILP:  0.06608724594116211
Benötigte Zeit von greedy:  0.008453369140625

Ausgewählte Anbieter von ILP:  [38, 46, 57]
Ausgewählte Anbieter von greedy:  [57, 38, 46]
Benötigte Zeit von ILP:  0.11505985260009766
Benötigte Zeit von greedy:  0.016468524932861328

Ausgewählte Anbieter von ILP:  [2, 3, 4, 13, 16, 17, 35, 43, 44, 46, 51, 57, 58]
Ausgewählte Anbieter von greedy:  [51, 38, 57, 42, 36, 2, 46, 17, 58]
Benötigte Zeit von ILP:  3.1074905395507812
Benötigte Zeit von greedy:  2.1359121799468994
Der Greedy-Algorithmus gibt höhren Preis zurück!

